https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#sphx-glr-download-tutorial-10-key-features-005-visualization-py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] ="expandable_segments:True" # seems to stop gpu mem from filling up despite clearing

In [3]:
import torch
import pandas as pd
from pathlib import Path
import optuna
from reprpo.hp.helpers import optuna_df

In [4]:
from reprpo.training import train
from reprpo.experiments import experiment_configs
from reprpo.hp.space import search_spaces

[2024-10-09 11:29:34,910] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## Objective

In [5]:
SEED=42
key_metric = "acc_gain_vs_ref/oos"
torch.manual_seed(SEED)
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
# silence please
import os
from loguru import logger
logger.remove()
logger.remove()
logger.add(os.sys.stderr, level="WARNING")

os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TQDM_DISABLE"] = "true"

In [7]:
f_db = f"sqlite:///optuna.db"
f = f_db.replace('sqlite:///', './')
print(f)
Path(f).parent.mkdir(parents=True, exist_ok=True)
f_db

./optuna.db


'sqlite:///optuna.db'

In [8]:
# print(f'to visualise run in cli\ncd nbs\noptuna-dashboard {f_db}')

In [9]:
from reprpo.hp.target import override, default_tuner_kwargs, list2tuples, objective
import functools
from reprpo.experiments import experiment_configs
import copy
import wandb

import optuna.pruners
from optuna_integration.wandb import WeightsAndBiasesCallback

## Opt

Note on pruning. It's only really usefull with validation metrics and for long jobs over many epochs. I've got a small proxy job so there is no need.

In [10]:
# from reprpo.experiments import experiment_configs
from reprpo.hp.space import experiment_configs
experiment_configs.keys()

dict_keys(['hs-svd-mse', 'hs-hra-rank', 'hs-ortho-prefvec', 'ether-prefvec', 'dpo', 'projbp', 'projgrad3'])

In [11]:
import warnings
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning) 

In [12]:
from optuna.study.study import storages, get_all_study_names
study_names = get_all_study_names(storage=f_db)

for study_name in study_names:
    print(study_name)
    study = optuna.load_study(study_name=study_name, storage=f_db)
    try:
        df_res = optuna_df(study, key_metric)
        display(df_res)
        print()
    except ValueError as e:
        print('-')

projgrad2


[W 2024-10-09 11:30:38,670] Study instance does not contain completed trials.


,importance,best
"projgrad2 N=✓0/315, best=nan",,



side-ether-prefvec


,importance,best
"side-ether-prefvec N=✓208/209, best=1.169",,
β,0.204,0.403787
use_angle_loss,0.157,True
use_dpo_loss,0.149,False
lr,0.131,0.000615
Htype,0.104,oft
weight_tokens,0.092,False
use_orth_loss,0.034,False
reduction,0.033,25
use_nll_loss,0.025,False


...

,importance,best
"hs-hra-rank N=✓259/262, best=1.152",,
lr,0.879,0.000333
β,0.069,0.37992
r,0.043,38
apply_GS,0.008,True
α,0.000,0.280164



ether-prefvec


,importance,best
"ether-prefvec N=✓321/326, best=1.183",,
lr,0.838,0.000378
β,0.074,1.984854
nb,0.020,20
Htype,0.017,ether
use_orth_loss,0.011,True
flip_side,0.009,True
use_angle_loss,0.008,True
reduction,0.007,1
use_proj_rel,0.005,True



projgrad3


,importance,best
"projgrad3 N=✓207/208, best=1.279",,
lr,0.943,0.000232
β,0.035,0.842679
scale_orth,0.009,False
mag_clip,0.005,float
weight_dim,0.005,1
neg_slope,0.002,0
reverse_pref,0.001,True
mag_clip_value,NaN,0.230096


In [13]:
# unit test, moved to pytest
# for exp_name, (N, trial2args) in search_spaces.items():
#     study = optuna.create_study(direction="maximize")
#     cfg = copy.deepcopy(experiment_configs[exp_name][1])
#     print('exp_name', exp_name)
#     for _ in range(10):
#         trial = study.ask()
#         kwargs = trial2args(trial)
#         override(cfg, default_tuner_kwargs)
#         override(cfg, kwargs)
#         kwargs = list2tuples(kwargs)

#     # try one dev run
#     kwargs['dev'] = True
#     _objective = functools.partial(objective, key_metric=key_metric, starter_experiment_name=exp_name, trial2args=trial2args)

#     study.optimize(_objective, 
#                 n_trials=1, # do 20 at a time, round robin, untill done
#                 gc_after_trial=True, 
#                 # catch=(AssertionError, OSError, RuntimeError, KeyError, torch.OutOfMemoryError)
#     )

In [14]:
MAX_TRIALS= 250
import numpy as np
spaces = list(search_spaces.items())
while True:
    np.random.shuffle(spaces)
    for exp_name, (max_trials, trial2args) in spaces:
        try:
            study_name = f"{exp_name}"
            study = optuna.create_study(
                study_name=study_name,
                direction="maximize",
                load_if_exists=True,
                storage=f_db,
                sampler=optuna.samplers.TPESampler(seed=SEED),
                # pruner=optuna.pruners.NopPruner(),
            )

            n = 0
            try:
                df = study.trials_dataframe().sort_values('value', ascending=False)
                n = len(df)
            except Exception as e:
                print(e)
                pass
            if n>0:
                print(f"loaded {n} {study_name} trials")

                df_res = optuna_df(study, key_metric)
                print(df_res.to_markdown())

            
            if n < max_trials:
                _objective = functools.partial(objective, key_metric=key_metric, starter_experiment_name=exp_name, trial2args=trial2args)

                study.optimize(_objective, 
                            n_trials=20, # do 20 at a time, round robin, untill done
                            gc_after_trial=True, 
                            catch=(AssertionError, OSError, RuntimeError, KeyError, torch.OutOfMemoryError)
                )

            print('='*80)
        except KeyboardInterrupt:
            break
        except Exception as e:
            logger.exception(e)

[I 2024-10-09 11:31:08,070] Using an existing study with name 'ether-prefvec' instead of creating a new one.


loaded 326 ether-prefvec trials
| ether-prefvec N=✓321/326, best=1.183   |   importance | best                   |
|:---------------------------------------|-------------:|:-----------------------|
| lr                                     |        0.845 | 0.00037772770210724844 |
| β                                      |        0.068 | 1.9848539330526844     |
| nb                                     |        0.04  | 20                     |
| reduction                              |        0.012 | 1                      |
| use_proj_rel                           |        0.006 | True                   |
| use_angle_loss                         |        0.005 | True                   |
| use_dpo_loss                           |        0.005 | False                  |
| collect_input                          |        0.004 | True                   |
| collect_hs                             |        0.004 | False                  |
| use_orth_loss                          |        0.003

[I 2024-10-09 11:34:59,341] Trial 326 finished with value: 0.43352601156069365 and parameters: {'lr': 0.0034974526946656434, 'collect_input': True, 'collect_hs': False, 'nb': 22, 'Htype': 'ether', 'flip_side': True, 'reduction': 1, 'β': 1.5570060506815473, 'use_orth_loss': True, 'use_angle_loss': True, 'use_dpo_loss': False, 'use_nll_loss': True, 'weight_tokens': True, 'use_proj_rel': False}. Best is trial 24 with value: 1.183044315992293.



| acc_inc/eval_ds [pp]                                                                                                                                                                                         |   train |    test |     oos |    rnd |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|--------:|--------:|-------:|
| ReprPO collect_input=True prefvec.use_nll_prefvec=True prefvec.use_orth_prefvec=True prefvec.weight_tokens=True prefvec.β=1.6 lr=0.0035 ether.Htype=ether ether.flip_side=True ether.nb=22 ether.reduction=1 | -45.455 | -40.945 | -56.647 | 10.526 |


[I 2024-10-09 11:38:15,721] Trial 327 finished with value: 1.1502890173410405 and parameters: {'lr': 0.0004977357947612501, 'collect_input': True, 'collect_hs': False, 'nb': 13, 'Htype': 'oft', 'flip_side': True, 'reduction': 13, 'β': 1.9971876761302187, 'use_orth_loss': False, 'use_angle_loss': True, 'use_dpo_loss': False, 'use_nll_loss': True, 'weight_tokens': True, 'use_proj_rel': True}. Best is trial 24 with value: 1.183044315992293.


...

| hs-hra-rank N=✓259/262, best=1.152   |   importance |         best |
|:-------------------------------------|-------------:|-------------:|
| lr                                   |        0.844 |  0.000332682 |
| β                                    |        0.072 |  0.37992     |
| r                                    |        0.042 | 38           |
| apply_GS                             |        0.042 |  1           |
| α                                    |        0     |  0.280164    |
loaded 261 hs-ortho-prefvec trials


[I 2024-10-10 04:15:41,527] Using an existing study with name 'dpo' instead of creating a new one.


| hs-ortho-prefvec N=✓259/261, best=1.152   |   importance | best                   |
|:------------------------------------------|-------------:|:-----------------------|
| lr                                        |        0.934 | 0.00041061394065629645 |
| β                                         |        0.022 | 1.970152011478674      |
| orthogonal_map                            |        0.014 | matrix_exp             |
| use_angle_loss                            |        0.014 | True                   |
| use_dpo_loss                              |        0.01  | False                  |
| weight_tokens                             |        0.003 | False                  |
| use_proj_rel                              |        0.002 | True                   |
| use_nll_loss                              |        0.001 | True                   |
| use_orth_loss                             |        0.001 | True                   |
loaded 250 dpo trials


[I 2024-10-10 04:15:42,857] Using an existing study with name 'projgrad3' instead of creating a new one.


| dpo N=✓248/250, best=1.276   |   importance |        best |
|:-----------------------------|-------------:|------------:|
| lr                           |            1 | 0.000264989 |
loaded 308 projgrad3 trials


## plot

In [ ]:
# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline

In [ ]:
search_spaces.keys()

In [ ]:
exp_name = 'projgrad'
trial2args = search_spaces[exp_name]

study_name = f"{exp_name}"
study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    load_if_exists=True,
    storage=f_db,
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.NopPruner(),
)
print('study.best_trial', study.best_trial)
df = study.trials_dataframe().query('state == "COMPLETE"').sort_values('value', ascending=False)
print(len(df))
plot_optimization_history(study)

In [13]:
# plot_timeline(study)

In [29]:
# plot_intermediate_values(study)

In [30]:
# plot_contour(study)


In [ ]:
plot_slice(study)


In [ ]:
plot_param_importances(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_parallel_coordinate(study)

### Apendix 1: dataclass 2 optuna

In [20]:
# import inspect
# import typing
# from typing import Literal

# def optuna_suggest_from_dataclass(t):
#     n = t.__name__
#     print(f'## {n}')
#     sig = inspect.signature(t)
#     for name, param in sig.parameters.items():
#         if param.annotation== bool:
#             print(f'"{name}": trial.suggest_categorical("{name}", [True, False]),')
#         elif param.annotation==int:
#             print(f'"{name}": trial.suggest_int("{name}", 1, 10),')
#         elif param.annotation ==float:
#             print(f'"{name}": trial.suggest_float("{name}", 0.1, 10.0),')
#         elif param.annotation == str:
#             print(f'"{name}": trial.suggest_categorical("{name}", ["a", "b", "c"]),')
#         elif param.annotation == tuple:
#             print(f'"{name}": trial.suggest_categorical("{name}", [(1, 2), (3, 4), (5, 6)]),')
#         elif typing.get_origin(param.annotation) == Literal:
#             print(f'"{name}": trial.suggest_categorical("{name}", {param.annotation.__args__}),')
#         else:
#             print(f"!!Unknown type {param}")
#             # print(name, param.default, param.annotation)

# optuna_suggest_from_dataclass(ReprPOConfig)
# for t in Transforms:
#     print(f'## {t}')
#     optuna_suggest_from_dataclass(t.value)
# for l in Losses:
#     print(f'## {l}')
#     optuna_suggest_from_dataclass(l.value)


# optuna_suggest_from_dataclass(DPOProjGradConfig)